<a href="https://colab.research.google.com/github/nithishdot/nithishdot/blob/main/ML_Assessment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files

uploded = files.upload()

Saving articles.csv to articles.csv


In [31]:
import pandas as pd
import numpy as np
import spacy
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import torch
from sentence_transformers import SentenceTransformer


In [12]:
df = pd.read_csv('articles.csv',encoding='latin-1')

In [13]:
df.head()

,Id,Heading,Article.Banner.Image,Outlets,Article.Description,Full_Article,Article_Type,Tonality
0,d6995462-5e87-453b-b64d-e9f1df6e94d2,"A Puzzling Maneuver, Then Freefall: NTSB Repor...",NaN,Essex Caller,<p>The helicopter that crashed in Southeast Al...,<p>The helicopter that crashed in Southeast Al...,Commercial,Negative
1,8b05e939-a89e-4548-b92b-013822e8ee7d,Bells Nexus Air Taxi Concept Rings Changes Fo...,NaN,Aviation Week Network,<p>A year after teasing the fledgling electric...,<p>A year after teasing the fledgling electric...,Commercial,Positive
2,69fcd400-bceb-4255-8277-619f2d68ac0b,Bell Helicopter Show Air Taxi Nexus,http://images.tmtpost.com/uploads/images/2019/...,TMTPost,<p>Bell released the full-size design of the v...,<p>Bell released the full-size design of the v...,Commercial,Positive
3,17943578-c11b-414b-b3f5-063d3a93157b,BELL DÉVOILE LA CONCEPTION INTÉGRALE DE SON TA...,http://www.fredzone.org/wp-content/uploads/201...,Fredzone,<p>Bell est une soci&eacute;t&eacute; am&eacut...,<p>Bell est une soci&eacute;t&eacute; am&eacut...,Commercial,Positive
4,f33c7b11-5f77-4a98-bb2e-d36689042aea,Les premiers retours dOlivier Ezratty,NaN,FrenchWeb,<p>It was still anecdotal to observe the explo...,<p>It was still anecdotal to observe the explo...,Commercial,Positive


In [9]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

In [16]:
def preprocess_text(text):
    text = re.sub(r'<.*?>', '', text) #HTML tags

    text = re.sub(r'[^\w\s]', '', text)     # Remove non-alphanumeric characters and extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if not token.is_stop]   # Lemmatization and remove stopwords

    return ' '.join(tokens)

In [17]:
df['cleaned_heading'] = df['Heading'].apply(preprocess_text)
df['cleaned_description'] = df['Article.Description'].apply(preprocess_text)
df['cleaned_full_article'] = df['Full_Article'].apply(preprocess_text)

In [21]:
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['Article_Type'])

In [22]:
df.head()

,Id,Heading,Article.Banner.Image,Outlets,Article.Description,Full_Article,Article_Type,Tonality,cleaned_heading,cleaned_description,cleaned_full_article,label
0,d6995462-5e87-453b-b64d-e9f1df6e94d2,"A Puzzling Maneuver, Then Freefall: NTSB Repor...",NaN,Essex Caller,<p>The helicopter that crashed in Southeast Al...,<p>The helicopter that crashed in Southeast Al...,Commercial,Negative,Puzzling Maneuver Freefall NTSB Report provide...,helicopter crash Southeast Alaska late Septemb...,helicopter crash Southeast Alaska late Septemb...,0
1,8b05e939-a89e-4548-b92b-013822e8ee7d,Bells Nexus Air Taxi Concept Rings Changes Fo...,NaN,Aviation Week Network,<p>A year after teasing the fledgling electric...,<p>A year after teasing the fledgling electric...,Commercial,Positive,Bells Nexus Air Taxi Concept Rings Changes evt...,year tease fledgling electric verticaltakeoffa...,year tease fledgling electric verticaltakeoffa...,0
2,69fcd400-bceb-4255-8277-619f2d68ac0b,Bell Helicopter Show Air Taxi Nexus,http://images.tmtpost.com/uploads/images/2019/...,TMTPost,<p>Bell released the full-size design of the v...,<p>Bell released the full-size design of the v...,Commercial,Positive,Bell Helicopter Air Taxi Nexus,Bell release fullsize design vertical takeoff ...,Bell release fullsize design vertical takeoff ...,0
3,17943578-c11b-414b-b3f5-063d3a93157b,BELL DÉVOILE LA CONCEPTION INTÉGRALE DE SON TA...,http://www.fredzone.org/wp-content/uploads/201...,Fredzone,<p>Bell est une soci&eacute;t&eacute; am&eacut...,<p>Bell est une soci&eacute;t&eacute; am&eacut...,Commercial,Positive,BELL DÉVOILE LA CONCEPTION INTÉGRALE DE SON TA...,bell e une socieacuteteacute ameacutericaine d...,bell e une socieacuteteacute ameacutericaine d...,0
4,f33c7b11-5f77-4a98-bb2e-d36689042aea,Les premiers retours dOlivier Ezratty,NaN,FrenchWeb,<p>It was still anecdotal to observe the explo...,<p>It was still anecdotal to observe the explo...,Commercial,Positive,les premier retour dOlivier Ezratty,anecdotal observe explosion passenger drone pr...,anecdotal observe explosion passenger drone pr...,0


In [43]:
X = df[['cleaned_heading', 'cleaned_description', 'cleaned_full_article']]
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [47]:
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

def encode_text(df):
    embeddings = []
    for index, row in df.iterrows():
        combined_text = f"{row['cleaned_heading']} {row['cleaned_description']} {row['cleaned_full_article']}"
        encoded_text = model.encode(combined_text, convert_to_tensor=True)
        embeddings.append(encoded_text)

    # Convert list of tensors to a 2D tensor
    embeddings_tensor = torch.stack(embeddings)
    return embeddings_tensor

# Generate embeddings for training and testing sets
X_train_embeddings = encode_text(X_train)
X_test_embeddings = encode_text(X_test)

# Ensure the shape of embeddings
print("Shape of X_train_embeddings:", X_train_embeddings.shape)
print("Shape of X_test_embeddings:", X_test_embeddings.shape)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Shape of X_train_embeddings: torch.Size([3444, 384])
Shape of X_test_embeddings: torch.Size([861, 384])


In [48]:
print("X_train_embeddings:", X_train_embeddings.shape)
print("y_train:", y_train.shape)
print("X_test_embeddings:", X_test_embeddings.shape)
print("y_test:", y_test.shape)


Shapes:
X_train_embeddings: torch.Size([3444, 384])
y_train: (3444,)
X_test_embeddings: torch.Size([861, 384])
y_test: (861,)


In [52]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

# SVM classifier
classifier = SVC()

# Train
classifier.fit(X_train_embeddings, y_train)


SVC()

In [50]:
from sklearn.model_selection import GridSearchCV

param_grid = {'C': [0.1, 1, 10], 'gamma': [0.001, 0.01, 0.1, 1]}
grid_search = GridSearchCV(SVC(), param_grid, cv=5)
grid_search.fit(X_train_embeddings, y_train)

# Get the best model
classifier = grid_search.best_estimator_


In [55]:
# Predict on the test set
y_pred = classifier.predict(X_test_embeddings)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Classification report
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

Accuracy: 0.9279907084785134


In [56]:
import joblib

# Save the model
joblib.dump(classifier, 'article_type_classifier.pkl')

# Load the model from disk
classifier = joblib.load('article_type_classifier.pkl')
